In [1]:
import geopandas as gpd
import os 
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
from unidecode import unidecode

In [2]:
#LIMITE_DEPARTAMENTAL_INEI_geogpsperu
os.chdir('C:/Users/FERNANDO/Documents/PI_INEQUIDAD/scripts_data/data/inputs/shape_files/Distrito')
gdf = gpd.read_file('DISTRITO.gpkg')
#gdf_1=gdf.iloc[:,[0,4,5,8]].copy()
#gdf_1
puntos_cap=gdf.to_crs(gdf.crs)


In [3]:
puntos_cap

,id,OBJECTID,UBIGEO,CCDD,CCPP,CCDI,NOMBDEP,NOMBPROV,NOMBDIST,CAPITAL,SHAPE_Length,SHAPE_Area,geometry
0,1,1,030220,03,02,20,APURIMAC,ANDAHUAYLAS,JOSE MARIA ARGUEDAS,HUANCABAMBA,0.570351,0.014649,"MULTIPOLYGON (((-73.37071 -13.70846, -73.36971..."
1,2,2,030415,03,04,15,APURIMAC,AYMARAES,TINTAY,TINTAY,0.706331,0.011908,"MULTIPOLYGON (((-73.17529 -13.80103, -73.17388..."
2,3,3,030409,03,04,09,APURIMAC,AYMARAES,LUCRE,LUCRE,0.553690,0.008678,"MULTIPOLYGON (((-73.24364 -13.81554, -73.24288..."
3,4,4,030214,03,02,14,APURIMAC,ANDAHUAYLAS,SAN MIGUEL DE CHACCRAMPA,CHACCRAMPA,0.402704,0.007104,"MULTIPOLYGON (((-73.63057 -13.90924, -73.62949..."
4,5,5,030206,03,02,06,APURIMAC,ANDAHUAYLAS,HUAYANA,HUAYANA,0.563448,0.007974,"MULTIPOLYGON (((-73.51096 -13.91122, -73.51029..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1886,1887,1887,250102,25,01,02,UCAYALI,CORONEL PORTILLO,CAMPOVERDE,CAMPOVERDE,2.101248,0.108873,"MULTIPOLYGON (((-74.77999 -8.25411, -74.77998 ..."
1887,1888,1888,250105,25,01,05,UCAYALI,CORONEL PORTILLO,YARINACOCHA,PUERTO CALLAO,1.230827,0.054303,"MULTIPOLYGON (((-74.61293 -8.052, -74.61237 -8..."
1888,1889,1889,250107,25,01,07,UCAYALI,CORONEL PORTILLO,MANANTAY,SAN FERNANDO,1.281366,0.052970,"MULTIPOLYGON (((-74.51857 -8.38629, -74.51795 ..."
1889,1890,1890,250201,25,02,01,UCAYALI,ATALAYA,RAIMONDI,ATALAYA,7.631965,1.201085,"MULTIPOLYGON (((-73.10633 -10.12237, -73.10587..."


In [4]:
capitales_distritales=puntos_cap.iloc[:,[6,7,8,9]]

In [5]:
capitales_distritales

,NOMBDEP,NOMBPROV,NOMBDIST,CAPITAL
0,APURIMAC,ANDAHUAYLAS,JOSE MARIA ARGUEDAS,HUANCABAMBA
1,APURIMAC,AYMARAES,TINTAY,TINTAY
2,APURIMAC,AYMARAES,LUCRE,LUCRE
3,APURIMAC,ANDAHUAYLAS,SAN MIGUEL DE CHACCRAMPA,CHACCRAMPA
4,APURIMAC,ANDAHUAYLAS,HUAYANA,HUAYANA
...,...,...,...,...
1886,UCAYALI,CORONEL PORTILLO,CAMPOVERDE,CAMPOVERDE
1887,UCAYALI,CORONEL PORTILLO,YARINACOCHA,PUERTO CALLAO
1888,UCAYALI,CORONEL PORTILLO,MANANTAY,SAN FERNANDO
1889,UCAYALI,ATALAYA,RAIMONDI,ATALAYA


In [6]:

import time

# Carga tu DataFrame
df = capitales_distritales.copy()

# Inicializa el geolocalizador
geolocator = Nominatim(user_agent="peru_district_capitals")

# Función de geocodificación con formato completo y fallback
def geolocalizar(row):
    intentos = [
        f"{row['CAPITAL']}, {row['NOMBDIST']}, {row['NOMBPROV']}, {row['NOMBDEP']}, Perú",
        f"{row['CAPITAL']}, {row['NOMBPROV']}, {row['NOMBDEP']}, Perú",
        f"{row['CAPITAL']}, {row['NOMBDEP']}, Perú",
        f"{row['CAPITAL']}, Perú"
    ]
    for intento in intentos:
        try:
            loc = geolocator.geocode(intento, timeout=10)
            if loc:
                return pd.Series([loc.latitude, loc.longitude, intento])
        except Exception as e:
            print(f"Error: {e}")
    return pd.Series([None, None, None])

# Aplica la geolocalización con espera
coords = []
for i, row in df.iterrows():
    resultado = geolocalizar(row)
    coords.append(resultado)
    print(f"{i+1}/{len(df)}: {row['CAPITAL']} → {resultado[2]}")
    time.sleep(1)

# Añade columnas al DataFrame
df[['lat', 'lon', 'consulta_usada']] = pd.DataFrame(coords, index=df.index)

1/1891: HUANCABAMBA → HUANCABAMBA, JOSE MARIA ARGUEDAS, ANDAHUAYLAS, APURIMAC, Perú
2/1891: TINTAY → TINTAY, TINTAY, AYMARAES, APURIMAC, Perú
3/1891: LUCRE → LUCRE, LUCRE, AYMARAES, APURIMAC, Perú
4/1891: CHACCRAMPA → CHACCRAMPA, SAN MIGUEL DE CHACCRAMPA, ANDAHUAYLAS, APURIMAC, Perú
5/1891: HUAYANA → HUAYANA, HUAYANA, ANDAHUAYLAS, APURIMAC, Perú
6/1891: CHAPIMARCA → CHAPIMARCA, CHAPIMARCA, AYMARAES, APURIMAC, Perú
7/1891: COTARUSE → COTARUSE, COTARUSE, AYMARAES, APURIMAC, Perú
8/1891: ANTABAMBA → ANTABAMBA, ANTABAMBA, ANTABAMBA, APURIMAC, Perú
9/1891: CARAYBAMBA → CARAYBAMBA, CARAYBAMBA, AYMARAES, APURIMAC, Perú
10/1891: HUAQUIRCA → HUAQUIRCA, HUAQUIRCA, ANTABAMBA, APURIMAC, Perú
11/1891: SABAINO → SABAINO, ANTABAMBA, APURIMAC, Perú
12/1891: COLCABAMBA → COLCABAMBA, AYMARAES, APURIMAC, Perú
13/1891: MOLLEBAMBA → MOLLEBAMBA, JUAN ESPINOZA MEDRANO, ANTABAMBA, APURIMAC, Perú
14/1891: SAN JUAN DE VIRUNDO → None
15/1891: CHALHUANCA → CHALHUANCA, CHALHUANCA, AYMARAES, APURIMAC, Perú
16/1891:

In [ ]:

df# acá identificamos que no existen valores que no tenga coordenadas, de todas formas hay algunos que me generan ruido
# ello por la forma en la que se obtuvieron
# 14/1891 SAN JUAN DE VIRUNDO -> None


,NOMBDEP,NOMBPROV,NOMBDIST,CAPITAL,lat,lon,consulta_usada
0,APURIMAC,ANDAHUAYLAS,JOSE MARIA ARGUEDAS,HUANCABAMBA,-13.734127,-73.351000,"HUANCABAMBA, JOSE MARIA ARGUEDAS, ANDAHUAYLAS,..."
1,APURIMAC,AYMARAES,TINTAY,TINTAY,-13.961597,-73.189943,"TINTAY, TINTAY, AYMARAES, APURIMAC, Perú"
2,APURIMAC,AYMARAES,LUCRE,LUCRE,-13.949472,-73.226189,"LUCRE, LUCRE, AYMARAES, APURIMAC, Perú"
3,APURIMAC,ANDAHUAYLAS,SAN MIGUEL DE CHACCRAMPA,CHACCRAMPA,-13.962438,-73.607960,"CHACCRAMPA, SAN MIGUEL DE CHACCRAMPA, ANDAHUAY..."
4,APURIMAC,ANDAHUAYLAS,HUAYANA,HUAYANA,-14.051177,-73.609013,"HUAYANA, HUAYANA, ANDAHUAYLAS, APURIMAC, Perú"
...,...,...,...,...,...,...,...
1886,UCAYALI,CORONEL PORTILLO,CAMPOVERDE,CAMPOVERDE,-8.558827,-74.812339,"CAMPOVERDE, CORONEL PORTILLO, UCAYALI, Perú"
1887,UCAYALI,CORONEL PORTILLO,YARINACOCHA,PUERTO CALLAO,-8.364807,-74.566811,"PUERTO CALLAO, YARINACOCHA, CORONEL PORTILLO, ..."
1888,UCAYALI,CORONEL PORTILLO,MANANTAY,SAN FERNANDO,-8.398268,-74.536905,"SAN FERNANDO, MANANTAY, CORONEL PORTILLO, UCAY..."
1889,UCAYALI,ATALAYA,RAIMONDI,ATALAYA,-10.729906,-73.754883,"ATALAYA, ATALAYA, UCAYALI, Perú"


In [21]:
df.to_excel("C:/Users/FERNANDO/Documents/PI_INEQUIDAD/scripts_data/data/outputs/distancias/cap_regionalxprovincias/lon_lat_cap_prov.xlsx", index=False)
